In [ ]:
! pip install streamlit -q

In [ ]:
!pip install pytube

In [ ]:
# changed client from default ANDROID_MUSIC
from pytube.innertube import _default_clients
_default_clients["ANDROID"] = _default_clients["WEB"]


In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
import torch
import torch.optim as optim
import itertools
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel, AutoTokenizer

# Load pre-trained DistilBERT model and tokenizer
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name) # DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertModel.from_pretrained(model_name)

def generate_text_embeddings(texts):
    # Tokenize input texts
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

    # Generate embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the embeddings from the last hidden layer
    embeddings = outputs.last_hidden_state[:, 0, :] # <---CLS # outputs.last_hidden_state.mean(dim=1)  # You can use other aggregation methods

    return embeddings

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
import torch
import torch.nn as nn
dropout = 0.1
class ImageProjection(nn.Module):
    def __init__(self, image_embedding_size, shared_embedding_size):
        super(ImageProjection, self).__init__()
        self.image_projection = nn.Linear(image_embedding_size, shared_embedding_size)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(shared_embedding_size, shared_embedding_size)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(shared_embedding_size)

    def forward(self, image_embeddings):
        projected_embeddings = self.image_projection(image_embeddings)

        x = self.gelu(projected_embeddings)
        x = self.fc(x)
        x = self.dropout(x)
        x = x + projected_embeddings
        x = self.layer_norm(x)

        return x # projected_embeddings

class TextProjection(nn.Module):
    def __init__(self, text_embedding_size, shared_embedding_size):
        super(TextProjection, self).__init__()
        self.text_projection = nn.Linear(text_embedding_size, shared_embedding_size)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(shared_embedding_size, shared_embedding_size)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(shared_embedding_size)

    def forward(self, text_embeddings):
        projected_embeddings = self.text_projection(text_embeddings)

        x = self.gelu(projected_embeddings)
        x = self.fc(x)
        x = self.dropout(x)
        x = x + projected_embeddings
        x = self.layer_norm(x)

        return x # projected_embeddings



In [ ]:
dropout = 0.1

def cross_entropy(preds, targets, reduction='none'):
    log_softmax = nn.LogSoftmax(dim=-1)
    loss = (-targets * log_softmax(preds)).sum(1)
    if reduction == "none":
        return loss
    elif reduction == "mean":
        return loss.mean()

temperature_value = torch.nn.Parameter(torch.tensor(1.0910))  # Initialize as a learnable parameter

def contrastive_clip_loss_function( text_projection,  image_projection, mode="eval" ):
    logits = (text_projection @ image_projection.T) / temperature_value
    if mode=="train":
        images_similarity = image_projection @ image_projection.T
        texts_similarity = text_projection @ text_projection.T
        targets = F.softmax( (images_similarity + texts_similarity) / 2 * temperature_value, dim=-1 )
        texts_loss = cross_entropy(logits, targets, reduction='none')
        images_loss = cross_entropy(logits.T, targets.T, reduction='none')
        loss =  (images_loss + texts_loss) / 2.0 # shape: (batch_size)
        return loss.mean()
    elif mode=="eval":
        return logits
    else:
        print("Mention mode")
        return None

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:

shared_embedding_size = 512
learning_rate = 5e-4

image_embedding_size=1024*2
resnet_model = models.resnet50(pretrained=True)
resnet_model = nn.Sequential( *list(resnet_model.children())[:-1] )
resnet_model.to(device)

max_length = 32
text_embedding_size = 768
model_name = 'distilbert-base-uncased'
text_tokenizer = DistilBertTokenizer.from_pretrained(model_name) # .to(device)
text_model = DistilBertModel.from_pretrained(model_name ).to(device)

image_projector = ImageProjection(image_embedding_size, shared_embedding_size).to(device)
text_projector = TextProjection(text_embedding_size, shared_embedding_size).to(device)

params = [{"params":resnet_model.parameters(), "lr":1e-4 }, {"params":text_model.parameters(), "lr":1e-5},
          # {"params":image_projector.parameters(), "lr": }, {"params":text_projector.parameters(), "lr":}
          {"params": itertools.chain( image_projector.parameters(), text_projector.parameters(), [temperature_value] ), "lr":1e-3 , "weight_decay":1e-3 }
         ]
optimizer = optim.AdamW( params,  weight_decay=0. )
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau( optimizer, mode="min", patience=2 , factor=0.8 )


In [35]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch

# Load the saved model checkpoint
checkpoint_path = "/content/drive/MyDrive/datasets/final_odel_from_Scratch_ResNet_DistilBERT.h5"
checkpoint = model = torch.load(checkpoint_path, map_location=torch.device('cpu'))


# Load the state dictionaries into the models and optimizer
resnet_model.load_state_dict(checkpoint['resnet_model_dict'])
text_model.load_state_dict(checkpoint['text_model_dict'])
image_projector.load_state_dict(checkpoint['image_projector_dict'])
text_projector.load_state_dict(checkpoint['text_projector_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
temperature_value_1 = nn.Parameter(torch.tensor(checkpoint['temperature_value']), requires_grad=True)

# Set the models to evaluation mode

image_projector.eval()




ImageProjection(
  (image_projection): Linear(in_features=2048, out_features=512, bias=True)
  (gelu): GELU(approximate='none')
  (fc): Linear(in_features=512, out_features=512, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
)

In [36]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.5 MB/s eta 0:00:00


In [37]:
from pytube import YouTube

In [38]:
from datasets import load_dataset

dataset = load_dataset("AlexZigma/msr-vtt")

README.md:   0%|          | 0.00/734 [00:00<?, ?B/s]

(…)-00000-of-00001-60e50ff5fbbd1bb5.parquet:   0%|          | 0.00/553k [00:00<?, ?B/s]

(…)-00000-of-00001-01bacdd7064306bc.parquet:   0%|          | 0.00/44.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6513 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/497 [00:00<?, ? examples/s]

In [39]:
dataset['val'][0]

{'video_id': 'video6513',
 'caption': 'a family is having coversation',
 'sen_id': 83560,
 'category': 14,
 'url': 'https://www.youtube.com/watch?v=A9pM9iOuAzM',
 'start time': 116.03,
 'end time': 126.21,
 'split': 'validate',
 'id': 6513,
 '__index_level_0__': 6128}

In [40]:
from pytube import YouTube

def download_video(video_url, output_filename):
    try:
        # Choose a video stream with resolution of 360p
        streams = YouTube(url=video_url).streams.filter(adaptive=True, subtype="mp4", resolution="360p", only_video=True)

        # Check if there is a valid stream
        if len(streams) == 0:
            raise Exception("No suitable stream found for this YouTube video!")

        # Download the video
        print("Downloading...")
        streams[0].download(filename=output_filename)
        print("Download completed.")
        return True
    except Exception as e:
        print(f"Error downloading video: {e}")
        return False





In [41]:
import cv2
import os

def extract_frames_with_timestamp(video_file):
    # Open the video file
    cap = cv2.VideoCapture(video_file)

    # Initialize lists to store frame data and timestamps
    frames_list = []
    timestamps_list = []

    # Initialize variables for frame extraction
    fps = cap.get(cv2.CAP_PROP_FPS)
    interval = int(fps)  # Extract one frame per second

    # Initialize frame counter
    frame_count = 0

    # Read frames until the end of the video
    while True:
        # Read a frame from the video
        ret, frame = cap.read()

        # Break the loop if no frame is read
        if not ret:
            break

        # Get the current timestamp (in milliseconds)
        timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)

        # Convert timestamp to seconds
        timestamp_sec = timestamp_ms / 1000.0

        # Check if it's time to extract a frame
        if frame_count % interval == 0:
            # Append frame and timestamp to the lists
            frames_list.append(frame)
            timestamps_list.append(timestamp_sec)

        # Increment frame counter
        frame_count += 1

    # Close the video file
    cap.release()
    print("Frames extracted:", len(frames_list))
    return frames_list, timestamps_list



In [42]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class FrameDataset(Dataset):
    def __init__(self, frame_list, image_size=224):
        self.frame_list = frame_list
        self.transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),  # Resize images to a consistent size
            transforms.ToTensor(),  # Convert images to tensors
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
        ])

    def __len__(self):
        return len(self.frame_list)

    def __getitem__(self, index):
        frame = self.frame_list[index]

        # Convert NumPy array to PIL Image
        frame_pil = Image.fromarray(frame)

        # Apply transformations
        frame_tensor = self.transform(frame_pil)

        return frame_tensor


In [43]:
resnet_model.eval()
image_projector.eval()
def new_create_image_embeddings(images):
    with torch.no_grad():
        image_embeddings = resnet_model(images)
        image_projection = image_projector(image_embeddings[:,:,0,0])
    return image_projection


def retrieve_frame(frame_dataset, input_query, n ):
    new_image_embeddings_list_train = []

    for index in tqdm(range(len( frame_dataset ))):
        images = frame_dataset[index]
        images = images.to(device)
        image_projection = new_create_image_embeddings(images.unsqueeze(0))
        new_image_embeddings_list_train.append( image_projection[0] )

    with torch.no_grad():
        inputs = tokenizer(input_query, return_tensors='pt', padding="max_length", max_length=max_length, truncation=True)
        inputs = inputs.to(device)
        outputs = text_model(**inputs)
        text_embeddings = outputs.last_hidden_state.mean(dim=1)
        text_projection = text_projector(text_embeddings)

    similarity_scores_list = []
    for index in tqdm(range(len(new_image_embeddings_list_train))):
        score = torch.dot( text_projection[0], new_image_embeddings_list_train[index] )
        similarity_scores_list.append( score.cpu().numpy() )

    max_indexes = np.array(similarity_scores_list).argsort()[-n:][::-1]

    return max_indexes




In [ ]:
import os
checkpoint_path = "/kaggle/input/temp1-432resnetbert/final_odel_from_Scratch_ResNet_DistilBERT.h5"

if not os.path.exists(checkpoint_path):
    print("Error: Checkpoint file not found!")
else:
    print("Checkpoint file found.")


In [44]:
import matplotlib.pyplot as plt

def display_frames_matching_query(video_url, query):
    # Download the video
    output_filename = "video.mp4"
    if not download_video(video_url, output_filename):
        print("Failed to download video.")
        return

    # Extract frames from the downloaded video
    video_file = "video.mp4"
    frames_list, timestamps_list = extract_frames_with_timestamp(video_file)

    # Retrieve frames matching the query
    frame_dataset = FrameDataset(frames_list)
    indexes = retrieve_frame(frame_dataset, query, n=5)
    selected_frames = [frames_list[i] for i in indexes]
    selected_timestamps = [timestamps_list[i] for i in indexes]

    # Display the frames matching the query and their timestamps
    print(f"Frames matching the query '{query}':")
    for frame, timestamp in zip(selected_frames, selected_timestamps):
        print(f"Timestamp: {timestamp}")
        # Display the frame as an image using matplotlib
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.axis('off')  # Turn off axis
        plt.show()



In [45]:
dataset['val'][10]

{'video_id': 'video6523',
 'caption': 'a man wearing a hoody dress removes the headset from his head',
 'sen_id': 108940,
 'category': 7,
 'url': 'https://www.youtube.com/watch?v=ylSh0dOsd8Q',
 'start time': 113.04,
 'end time': 125.09,
 'split': 'validate',
 'id': 6523,
 '__index_level_0__': 6139}

In [46]:
first_row = dataset['val'][10]
video_url = first_row['url']
start_time = first_row['start time']
end_time = first_row['end time']
query = first_row['caption']
# Call the function
display_frames_matching_query(video_url, query)

Error downloading video: HTTP Error 403: Forbidden
Failed to download video.


In [47]:
video_url= ""
query=""
display_frames_matching_query(video_url, query)

Error downloading video: regex_search: could not find match for (?:v=|\/)([0-9A-Za-z_-]{11}).*
Failed to download video.


In [48]:
!wget -q -O - ipv4.icanhazip.com

34.139.135.196


In [49]:
%%writefile app.py
import streamlit as st
from pytube.innertube import _default_clients
import numpy as np
import cv2
from PIL import Image
import torch
import matplotlib.pyplot as plt
from torchvision.models import resnet50
from transformers import DistilBertTokenizer, DistilBertModel
from datasets import load_dataset
from tqdm import tqdm
import psutil
_default_clients["ANDROID"] = _default_clients["WEB"]
import time
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
import torch
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import time
import torch
import torch.nn as nn
from pytube import YouTube
from datasets import load_dataset
import torch.nn.functional as F
import torch.optim as optim
import itertools
import cv2
import os
import cv2
from transformers import DistilBertTokenizer, DistilBertModel
from torchvision.models import resnet50
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


from transformers import DistilBertTokenizer, DistilBertModel, AutoTokenizer

# Load pre-trained DistilBERT model and tokenizer
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name) # DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertModel.from_pretrained(model_name)

def generate_text_embeddings(texts):
    start_time = time.time()  # Start time for text embedding
    # Tokenize input texts
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    # Generate embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    # Get the embeddings from the last hidden layer
    embeddings = outputs.last_hidden_state[:, 0, :]  # <---CLS # outputs.last_hidden_state.mean(dim=1)  # You can use other aggregation methods
    end_time = time.time()  # End time for text embedding
    elapsed_time = end_time - start_time  # Calculate elapsed time
    return embeddings, elapsed_time


# Define ResNet-BERT models
class ImageProjection(nn.Module):
    def __init__(self, image_embedding_size, shared_embedding_size):
        super(ImageProjection, self).__init__()
        self.image_projection = nn.Linear(image_embedding_size, shared_embedding_size)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(shared_embedding_size, shared_embedding_size)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(shared_embedding_size)

    def forward(self, image_embeddings):
        projected_embeddings = self.image_projection(image_embeddings)

        x = self.gelu(projected_embeddings)
        x = self.fc(x)
        x = self.dropout(x)
        x = x + projected_embeddings
        x = self.layer_norm(x)

        return x

class TextProjection(nn.Module):
    def __init__(self, text_embedding_size, shared_embedding_size):
        super(TextProjection, self).__init__()
        self.text_projection = nn.Linear(text_embedding_size, shared_embedding_size)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(shared_embedding_size, shared_embedding_size)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(shared_embedding_size)

    def forward(self, text_embeddings):
        projected_embeddings = self.text_projection(text_embeddings)

        x = self.gelu(projected_embeddings)
        x = self.fc(x)
        x = self.dropout(x)
        x = x + projected_embeddings
        x = self.layer_norm(x)

        return x


dropout = 0.1

def cross_entropy(preds, targets, reduction='none'):
    log_softmax = nn.LogSoftmax(dim=-1)
    loss = (-targets * log_softmax(preds)).sum(1)
    if reduction == "none":
        return loss
    elif reduction == "mean":
        return loss.mean()

temperature_value = torch.nn.Parameter(torch.tensor(1.0910))  # Initialize as a learnable parameter

def contrastive_clip_loss_function( text_projection,  image_projection, mode="eval" ):
    logits = (text_projection @ image_projection.T) / temperature_value
    if mode=="train":
        images_similarity = image_projection @ image_projection.T
        texts_similarity = text_projection @ text_projection.T
        targets = F.softmax( (images_similarity + texts_similarity) / 2 * temperature_value, dim=-1 )
        texts_loss = cross_entropy(logits, targets, reduction='none')
        images_loss = cross_entropy(logits.T, targets.T, reduction='none')
        loss =  (images_loss + texts_loss) / 2.0 # shape: (batch_size)
        return loss.mean()
    elif mode=="eval":
        return logits
    else:
        print("Mention mode")
        return None


shared_embedding_size = 512
learning_rate = 5e-4

image_embedding_size=1024*2
resnet_model = models.resnet50(pretrained=True)
resnet_model = nn.Sequential( *list(resnet_model.children())[:-1] )
resnet_model.to(device)

max_length = 32
text_embedding_size = 768
model_name = 'distilbert-base-uncased'
text_tokenizer = DistilBertTokenizer.from_pretrained(model_name) # .to(device)
text_model = DistilBertModel.from_pretrained(model_name ).to(device)

image_projector = ImageProjection(image_embedding_size, shared_embedding_size).to(device)
text_projector = TextProjection(text_embedding_size, shared_embedding_size).to(device)

params = [{"params":resnet_model.parameters(), "lr":1e-4 }, {"params":text_model.parameters(), "lr":1e-5},
          # {"params":image_projector.parameters(), "lr": }, {"params":text_projector.parameters(), "lr":}
          {"params": itertools.chain( image_projector.parameters(), text_projector.parameters(), [temperature_value] ), "lr":1e-3 , "weight_decay":1e-3 }
         ]
optimizer = optim.AdamW( params,  weight_decay=0. )
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau( optimizer, mode="min", patience=2 , factor=0.8 )

# Load the saved model checkpoint
checkpoint_path = "/content/drive/MyDrive/datasets/final_odel_from_Scratch_ResNet_DistilBERT.h5"
checkpoint = model = torch.load(checkpoint_path, map_location=torch.device('cpu'))


# Load the state dictionaries into the models and optimizer
resnet_model.load_state_dict(checkpoint['resnet_model_dict'])
text_model.load_state_dict(checkpoint['text_model_dict'])
image_projector.load_state_dict(checkpoint['image_projector_dict'])
text_projector.load_state_dict(checkpoint['text_projector_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Set the models to evaluation mode

image_projector.eval()
resnet_model.eval()
text_model.eval()
text_projector.eval()




dataset = load_dataset("AlexZigma/msr-vtt")


def download_video(video_url, output_filename):
    try:
        # Choose a video stream with resolution of 360p
        streams = YouTube(url=video_url).streams.filter(adaptive=True, subtype="mp4", resolution="360p", only_video=True)

        # Check if there is a valid stream
        if len(streams) == 0:
            raise Exception("No suitable stream found for this YouTube video!")

        # Download the video
        print("Downloading...")
        streams[0].download(filename=output_filename)
        print("Download completed.")
        st.write("Download completed.")
        # st.write("Frames extracted:", len(frames_list))
        return True
    except Exception as e:
        print(f"Error downloading video: {e}")
        st.write(f"Error downloading video: {e}")
        return False


class FrameDataset(Dataset):
    def __init__(self, frame_list, image_size=224):
        self.frame_list = frame_list
        self.transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),  # Resize images to a consistent size
            transforms.ToTensor(),  # Convert images to tensors
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
        ])

    def __len__(self):
        return len(self.frame_list)

    def __getitem__(self, index):
        frame = self.frame_list[index]

        # Convert NumPy array to PIL Image
        frame_pil = Image.fromarray(frame)

        # Apply transformations
        frame_tensor = self.transform(frame_pil)

        return frame_tensor

def extract_frames_with_timestamp(video_file):
    # Open the video file
    cap = cv2.VideoCapture(video_file)

    # Initialize lists to store frame data and timestamps
    frames_list = []
    timestamps_list = []

    # Initialize variables for frame extraction
    fps = cap.get(cv2.CAP_PROP_FPS)
    interval = int(fps)  # Extract one frame per second

    # Initialize frame counter
    frame_count = 0

    # Read frames until the end of the video
    while True:
        # Read a frame from the video
        ret, frame = cap.read()

        # Break the loop if no frame is read
        if not ret:
            break

        # Get the current timestamp (in milliseconds)
        timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)

        # Convert timestamp to seconds
        timestamp_sec = timestamp_ms / 1000.0

        # Check if it's time to extract a frame
        if frame_count % interval == 0:
            # Append frame and timestamp to the lists
            frames_list.append(frame)
            timestamps_list.append(timestamp_sec)

        # Increment frame counter
        frame_count += 1

    # Close the video file
    cap.release()
    print("Frames extracted:", len(frames_list))
    st.write("Frames extracted:", len(frames_list))
    return frames_list, timestamps_list



# Define function to retrieve frames matching the query
def new_create_image_embeddings(images):
    # start_time = time.time()  # Start time for image embedding
    with torch.no_grad():
        image_embeddings = resnet_model(images)
        image_projection = image_projector(image_embeddings[:,:,0,0])
    # end_time = time.time()  # End time for image embedding
    # elapsed_time = end_time - start_time  # Calculate elapsed time
    return image_projection



def retrieve_frame(frame_dataset, input_query, n ):
    new_image_embeddings_list_train = []
    print('image embedding')
    start_time = time.time()
    for index in tqdm(range(len( frame_dataset ))):
        images = frame_dataset[index]
        images = images.to(device)
        image_projection = new_create_image_embeddings(images.unsqueeze(0))
        new_image_embeddings_list_train.append(image_projection[0])
    end_time = time.time()  # End time for image embedding
    elapsed_time = end_time - start_time  # Calculate elapsed time
    print(f"Time taken for image embedding : {elapsed_time:.2f} seconds")
    print('text embedding')
    start_time = time.time()
    with torch.no_grad():
        inputs = tokenizer(input_query, return_tensors='pt', padding="max_length", max_length=max_length, truncation=True)
        inputs = inputs.to(device)
        outputs = text_model(**inputs)
        text_embeddings = outputs.last_hidden_state.mean(dim=1)
        text_projection = text_projector(text_embeddings)
    end_time = time.time()  # End time for image embedding
    elapsed_time = end_time - start_time  # Calculate elapsed timeprint(f"Time taken for text embedding: {text_elapsed_time:.2f} seconds")
    print('similarity calculate')
    similarity_scores_list = []
    for index in tqdm(range(len(new_image_embeddings_list_train))):
        score = torch.dot( text_projection[0], new_image_embeddings_list_train[index] )
        similarity_scores_list.append( score.cpu().numpy() )

    max_indexes = np.array(similarity_scores_list).argsort()[-n:][::-1]

    return max_indexes
# Add performance metrics function
def get_performance_metrics(frames_processed, start_time, end_time, download_start_time, download_end_time):
    # Calculate elapsed time for extraction
    extraction_time = end_time - start_time

    # Calculate elapsed time for downloading
    download_time = download_end_time - download_start_time

    # Calculate downloading speed (MB/s)
    if download_time > 0:
        download_speed = (os.path.getsize(video_filename) / 1024 ** 2) / download_time
        st.write(f"Downloading Speed: {download_speed} MB/s")

    # Calculate extraction speed (frames/s)
    if extraction_time > 0:
        extraction_speed = frames_processed / extraction_time
        st.write(f"Extraction Speed: {extraction_speed} frames/s")

    # CPU usage
    cpu_percent = psutil.cpu_percent()
    st.write(f"CPU Usage: {cpu_percent}%")

    # Memory usage
    memory = psutil.virtual_memory()
    memory_usage = memory.used / 1024 ** 2  # Convert to MB
    st.write(f"Memory Usage: {memory_usage} MB")

    # Frames processed
    st.write(f"Frames Processed: {frames_processed}")

    # Calculate frames per second (FPS)
    if extraction_time > 0:
        fps = frames_processed / extraction_time
        st.write(f"Frames per Second (FPS): {fps}")


# Streamlit App
st.title("Video Frame Extractor")

# Option to input YouTube link and query
video_url = st.text_input("Enter YouTube Video Link")
query = st.text_input("Enter Query")

# if st.button("Calculate Image Embeddings"):
#     if video_url:
#         download_start_time = time.time()

#         # Download video
#         video_filename = "video.mp4"
#         if download_video(video_url, video_filename):
#             # End download time
#             download_end_time = time.time()

#             # Display downloaded video
#             st.video(video_filename)

#             # Extract frames
#             frames, timestamps = extract_frames_with_timestamp(video_filename)
#             frame_dataset = FrameDataset(frames)

#             # Calculate image embeddings
#             image_embeddings = new_create_image_embeddings(frame_dataset)

#             # Save image embeddings for later use
#             st.cache(image_embeddings, allow_output_mutation=True, show_spinner=False)

#             st.write("Image embeddings calculated successfully.")

#     else:
#         st.write("Please enter YouTube video link.")

# if st.button("Extract Frames"):
#     if query:
#         # Retrieve cached image embeddings
#         image_embeddings = st.cache(image_embeddings, allow_output_mutation=True)

#         # Start extraction time
#         start_time = time.time()

#         # Retrieve frames matching query
#         selected_indexes = retrieve_frame(frame_dataset, query, n=5)
#         selected_frames = [frames[i] for i in selected_indexes]
#         selected_timestamps = [timestamps[i] for i in selected_indexes]

#         # End extraction time
#         end_time = time.time()

#         # Display selected frames
#         for frame, timestamp in zip(selected_frames, selected_timestamps):
#             st.write(f"Timestamp: {timestamp}")
#             st.image(frame, use_column_width=True)

#         # Display video with marked timestamps
#         st.video(video_filename, start_time=timestamps[selected_indexes[0]])
#         for timestamp in selected_timestamps:
#             st.video(video_filename, start_time=timestamp)

#         # Performance metrics
#         get_performance_metrics(len(frames), start_time, end_time, download_start_time, download_end_time)

#     else:
#         st.write("Please enter query.")


if st.button("Extract Frames"):
    if video_url and query:
        download_start_time = time.time()

        # Download video
        video_filename = "video.mp4"
        if download_video(video_url, video_filename):
            # End download time
            download_end_time = time.time()

            # Display downloaded video
            st.video(video_filename)

            # Extract frames
            frames, timestamps = extract_frames_with_timestamp(video_filename)
            frame_dataset = FrameDataset(frames)

            # Start extraction time
            start_time = time.time()

            # Retrieve frames matching query
            selected_indexes = retrieve_frame(frame_dataset, query, n=5)
            selected_frames = [frames[i] for i in selected_indexes]
            selected_timestamps = [timestamps[i] for i in selected_indexes]
            print('step')
            # End extraction time
            end_time = time.time()
            progress_bar = st.progress(0)
            num_frames = len(frames)
            for frame, timestamp in zip(selected_frames, selected_timestamps):
                st.write(f"Timestamp: {timestamp}")
                st.image(frame, use_column_width=True)



              # Redisplay video with marked timestamps on progress bar
            st.video(video_filename, start_time=timestamps[selected_indexes[0]])
            for timestamp in selected_timestamps:
                st.video(video_filename, start_time=timestamp)



            # Performance metrics
            get_performance_metrics(len(frames), start_time, end_time, download_start_time, download_end_time)

    else:
        st.write("Please enter YouTube video link and query.")

Writing app.py


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼⠴⠦